<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4-4 Mechanisms of the AutoGraph

Herer is the mechanisms of Autograph

## 1. Machanism of Autograph

**What heppens when we define a function using decorator `@tf.function`?**

In [3]:
import tensorflow as tf
import numpy as np

@tf.function(autograph=True)
def myadd(a, b):
  for i in tf.range(3):
    tf.print(i)

  c = a + b

  print("tracing")
  return c

Nothing heppens except a function signature is recorded in the stack of Python.

**What heppens when the function decoraded by `@tf.function` is called?**

In [5]:
myadd(tf.constant('hello'), tf.constant('World'))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloWorld'>

There are two indicent:

1. The graph is created.
A static graph is created. The python code inside the function is executed, the tensor type of each variable is determined, and the operator is added to the graph according to the order of execution. During this period, if the argument `autograph=True` (defaut) is setted, convertting of the controlling flow in Python to the one inside TensorFlow graph will happen. The majority of the work are:
- replacing `if` to `tf.cond` operator
- replacing `while` and `for` looping to `tf.while_loop`
- when necessary, add `tf.control_dependencies` to specify the dependencies of executing orders.

This is identical to the following expression in TensorFlow 1.X:

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

g = tf.Graph()

with g.as_default():
  a = tf.placeholder(shape=[], dtype=tf.string)
  b = tf.placeholder(shape=[], dtype=tf.string)

  cond = lambda i: i < tf.constant(3)

  def body(i):
    tf.print(i)
    return (i + 1)

  loop = tf.while_loop(cond, body, loop_vars=[0])
  loop

  with tf.control_dependencies(loop):
    c = tf.strings.join([a, b])

  print('tracing')

2. Execution of the graph.
This is identical to the following expressions in TensorFlow 1.X:

In [ ]:
with tf.Session(graph=g) as sess:
  sess.run(c, feed_dict={a: tf.constant('hello'), b: tf.constant('world')})

So the results for the first step comes first: A string "tracing" printed by the standard I/O stream of Python

And next is the result of the second step: A string "1,2,3" printed by standard I/O stream of TensorFlow.

**What is going to heppen when we call this function again with the same types of the input arguments?**

In [11]:
myadd(tf.constant('good'), tf.constant('morning'))

<tf.Tensor 'StatefulPartitionedCall_2:0' shape=() dtype=string>

Only one thing heppens: execution of the graph, which is the second step mentioned above.

So the string "tracing" doesnt appear.

** What is going to happen when we call this function again with some different types of the input arguments?**

In [13]:
myadd(tf.constant(1), tf.constant(2))

tracing


<tf.Tensor 'StatefulPartitionedCall_3:0' shape=() dtype=int32>

Since the data type of the argument has been changed, the previously created graph cannot be used again.

Tow more tasks to be done: create new graph and execute it.

The result of the first step will be observed again, i.e. a string "tracing" printed by the standard I/O stream of Python.

Ans next is the result of the second step: A string "1,2,3" is printed by the standard I/O stream of TensorFlow.

**If the data type of the argument is not Tensor in the original definition of this function, then the graph will bw re-created each time adter calling this function.**

The demonstrated code below re-created graph every time, so it is recommended to use Tensor type as the arguments when calling the function decoraded by `@tf.function`.

In [14]:
myadd('hello', 'world')
myadd('good', 'morning')

tracing
tracing


<tf.Tensor 'StatefulPartitionedCall_5:0' shape=() dtype=string>

## 2. Scrutinize the Coding Rules of Autograph Again

Have better understanding to the three rulse of coding of Autograph after knowing the mechanism.

1. Should use the TensorFlow-defined function to be decorated by `@tf.function` as mush as possible, instead of those Python functions. For instance, `tf.print` should be use instead of `print`.

Explanations: Python function are only used during the stage of creating static graph. The Python functions are not able to be embedded into the static graph, so these Python functions are not calculated during the calling after the graph creation; in contras, TensorFlow functions are able to be embedded into the graph. Using Python function causing unmatched outputs between the "eager execution" before the decoration by `@tf.function` and the "execution of static graph" after the decoration by `@tf.function`.

2. Avoid defining `tf.Variable` inside the decorator `@tf.function`.

Explanations: The defined `tf.Variable` will be re-created every time when calling the function during the "eager execution" stage. However, this re-creation of `tf.Variable` only takes place at the first step, i.e. tracing Python code to the create the graph, which is introducing unmatched outputs between the "eager execution" before the decoration by `@tf.function` and the "execution of static graph" after the decoration by `@tf.function`. In fact, TensorFlow throws error in most of such cases.

3. Functions that are decorated by `@tf.function` cannot modify the variables outside the functions with the data types such as Python list, dictionary, etc.

Explanation: Static graph is executed in the TensorFlow kernels, which are compiled from C++ code, thus the list and dictionary in Python are not able to be embedded into the graph. These data types can only be read during the stage of graph creating and cannot be modified during the graph execution.